<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Trees/SAT_Construction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAT Constructions 2

The purpose of this Python notebook is to investigate the SAT construction that tests if a graph is connected.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and write code to complete the functions.

You can save a local copy of this notebook in your Google account and work through it in Colab (recommended) or you can download the notebook and run it locally using Jupyter notebook or similar. If you are using CoLab, we recommend that turn off AI autocomplete (under cog icon in top-right corner), which will give you the answers and defeat the purpose of the exercise.

A fully working version of this notebook with the complete answers can be found [here](https://github.com/udlbook/udlbook/blob/main/Trees/SAT_Construction2_Answers.ipynb).

Contact me at iclimbtreesmail@gmail.com if you find any mistakes or have any suggestions.

In [ ]:
!pip install z3-solver
from z3 import *
import numpy as np
from itertools import combinations

# Connected components

Finally, let's develop a construction that tells us if the elements of an undirected graph are connected.   Consider the two adjacency matrices:

\begin{equation}
A_1 = \begin{bmatrix}1 & 1 & 0 & 0 & 0 & 0 & 0 & 0  \\
                     1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 \\
                     0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 \\
                     0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
                     0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 \\
                     0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 \\
                     0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
                     0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 \\
 \end{bmatrix} \quad\quad
 A_2 = \begin{bmatrix}1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 \\
                      1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 \\
                      0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 \\
                      1 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
                      0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 \\
                      0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 \\
                      0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 \\
                      0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 \\
 \end{bmatrix}
\end{equation}

Each matrix represents the edges in a graph containing eight nodes.  Elements $(i,j)$ and $(j,i)$ will be set to one if there is an edge between nodes $i$ and $j$. The diagonal elements are all set to one.

For matrix $A_{1}$ the nodes are all connected;  node 1 connects to node 2, which connects to node 3, and so on up to node 8.   

For matrix $A_{2}$ however, the nodes are not all connected.  Nodes 7 and 8 are connected to each other but not connected any of the other nodes.


# Test for connected components

We can test for the connectivity of the graph implied by an $8\times 8$ matrix $\mathbf{A}$ by computing new adjacency matrices $\mathbf{B},\mathbf{C},\mathbf{D}$ where:

$$\begin{aligned}
B_{ij} &\Leftrightarrow \bigvee_k (A_{ik} \land A_{kj})\\
C_{ij} &\Leftrightarrow \bigvee_k (B_{ik} \land B_{kj})\\
D_{ij} &\Leftrightarrow \bigvee_k (C_{ik} \land C_{kj})
\end{aligned}
$$

and then enforcing the constraint that the first row of $\mathbf{D}$ contains all true elements:

$$ \bigvee_k D_{0,k}$$

In general, if the initial matrix $\mathbf{A}$ is of size $N\times N$, we will need to compute $\log_2[N]$ intermediate matrices $\mathbf{B},\mathbf{C},\mathbf{D}$.

Now let's write a SAT routine to check if an adjacency matrix represents a fully-connected graph

In [ ]:
def is_fully_connected(s, adjacency):
  # Size of the adjacency matrix
  n_components = len(adjacency)
  # We'll construct a N x N x log[N] array of variables
  # The NxN variables in the first layer represent A, the variables in the second layer represent B and so on
  n_layers = math.ceil(math.log(n_components,2))+1
  connected = [[[ z3.Bool("conn_{%d,%d,%d}"%((i,j,n))) for n in range(0, n_layers)] for j in range(0, n_components) ] for i in range(0, n_components) ]

  # Constraint 1
  # The value in the top layer of the connected structure is equal to the adjacency matrix
  # TODO -- replace this line
  s.add(connected[0][0][0])

  # Constraint 2
  # Value at position [i,j] in layer n is true if there is a k such that position[i,k] and positions [k,j] in layer n-1 are true
  for n in range(1,n_layers):
    for i in range(n_components):
      for j in range(n_components):
        # TODO -- replace this line
        s.add(connected[i][j][n])

  # Constraint 3 -- any row of column of the matrix should be full of ones at the end (everything is connected)
  # TODO -- replace this line
  s.add(connected[0][0][n_layers-1])

  return s

Finally, let's write a routine that tests the adjacency of the two matrices above using a SAT solver.

In [ ]:
def test_is_fully_connected(A):
  # Set up the SAT solver
  s = Solver()

  # Convert the input matrix to z3 Boolean values
  n_components = A.shape[0]
  adjacency= [[ z3.Bool("a_{%d,%d}"%((i,j)))  for j in range(0, n_components) ] for i in range(0, n_components) ]
  for i in range(n_components):
    for j in range(n_components):
     if A[i,j]!=0:
        s.add(adjacency[i][j])
     else:
        s.add(Not(adjacency[i][j]))

  # Run the routine
  s = is_fully_connected(s, adjacency)

  # Check if it's SAT (creates the model)
  sat_result = s.check()
  print(sat_result)

  # If it was SAT then print out the layers of the 3D structure
  if sat_result == z3.sat:
    result = s.model()
    c_vals = np.array([[[int(bool(result[z3.Bool("conn_{%d,%d,%d}" % (i, j,n))])) for n in range(0, n_components-1)] for j in range(0,n_components) ] for i in range(0,n_components) ] )
    for n in range(math.ceil(math.log(n_components,2))+1):
      print("Layer:",n)
      print(c_vals[:,:,n])

In [ ]:
A1 = np.array([[1, 1, 0, 0, 0, 0, 0, 0],
               [1, 1, 1, 0, 0, 0, 0, 0],
               [0, 1, 1, 1, 0, 0, 0, 0],
               [0, 0, 1, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 1, 1, 0, 0],
               [0, 0, 0, 0, 1, 1, 1, 0],
               [0, 0, 0, 0, 0, 1, 1, 1],
               [0, 0, 0, 0, 0, 0, 1, 1]])
test_is_fully_connected(A1)

In [ ]:
A2 = np.array([[1, 1, 0, 1, 0, 0, 0, 0],
               [1, 1, 1, 0, 0, 0, 0, 0],
               [0, 1, 1, 1, 0, 0, 0, 0],
               [1, 0, 1, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 1, 1, 0, 0],
               [0, 0, 0, 0, 1, 1, 0, 0],
               [0, 0, 0, 0, 0, 0, 1, 1],
               [0, 0, 0, 0, 0, 0, 1, 1]])
test_is_fully_connected(A2)